<a href="https://colab.research.google.com/github/mateopolancecofficial/DeepLearningProjects/blob/main/ComputerVision/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
# set installation txt file with all relevant packages
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import pathlib
import shutil
import random
import imghdr

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [76]:
tf.debugging.set_log_device_placement(True)

if tf.config.list_physical_devices('GPU'):
  if len(tf.config.list_physical_devices('GPU')) > 1:
    strategy = tf.distribute.MirroredStrategy()
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
  else:
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
# download images folder
#!wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
#from shutil import unpack_archive
#unpack_archive('/content/flower_photos.tgz', '/content/')

--2021-06-11 20:41:40--  https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228813984 (218M) [application/x-compressed-tar]
Saving to: ‘flower_photos.tgz’

flower_photos.tgz   100%[===================>] 218.21M   191MB/s    in 1.1s    

2021-06-11 20:41:42 (191 MB/s) - ‘flower_photos.tgz’ saved [228813984/228813984]



In [113]:
# load data parameters

MAX_WIDTH, MAX_HEIGHT = 100, 100
origin = os.path.dirname(os.path.abspath("ImageClassification.ipynb"))
print(origin)
train_path = 'flower_photos'
#test_path = ''
untar = False
color_mode = "rgb"
train_source_folder_path = os.path.join(origin, train_path)
#test_source_folder_path = os.path.join(origin, test_path)
print(source_folder_path)

batch_size = 32
test_split = 0.2
validation_split = 0.2
seed = 100

image_extesions = ['png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif']


/content
/content/flower_photos


In [114]:
# check file types from all directories


In [118]:
# split dataset on test and train if is not currently

def train_test_split(origin, train_path, test_split):

  train_path = os.path.join(origin, train_path)
  test_path = "test_data"
  test_path = os.path.join(origin, test_path)

  if os.path.exists(test_path):
    shutil.rmtree(test_path)
  os.mkdir(test_path)

  for dir in os.listdir(train_path):
    files = os.listdir(os.path.join(train_path, dir))
    if len(files) > 0:
      if not os.path.exists(os.path.join(test_path, dir)):
        os.mkdir(os.path.join(test_path, dir))
      test_num = len(files) * test_split
      print(test_num)
      indicies = random.sample(range(len(files)), round(test_num))
      test_files_names = [files[i] for i in indicies]
      # move file from train dataset to test dataset
      for f in test_files_names:
        image_type = imghdr.what(os.path.join(os.path.join(train_path, dir), f))
        #print(image_type)
        if image_type in image_extesions:
          os.replace(os.path.join(os.path.join(train_path, dir), f), 
                     os.path.join(os.path.join(test_path, dir), f))


In [119]:
train_test_split(origin, train_path, test_split)

139.8
159.8
126.60000000000001
128.20000000000002
179.60000000000002


In [63]:
def get_min_img_size(source_folder_path):
  """
  Check if all images are of the same size and get min image width and min image
  height.
  param source_folder_path:      path of a source folder
  return img_width, img_height:  min image width and height scalar values
  """

  img_width, img_height = None, None
  dirs = os.listdir(source_folder_path)

  for dir in dirs:
    files = os.listdir(source_folder_path + "/" + dir)
    for f in files:
      image_path = source_folder_path + "/" + dir + "/" + f
      image = tf.keras.preprocessing.image.load_img(image_path)
      image = keras.preprocessing.image.img_to_array(image)
      if img_width == None:
        img_width, img_height = image.shape[0], image.shape[1]
      else:
        if image.shape[0] < img_width:
          img_width = image.shape[0]
        if image.shape[1] < img_height:
          img_height = image.shape[1]
  
  return img_width, img_height

In [64]:
def set_resize_image_parameters(img_width, img_height):
  """
  Set parameters for resizing input images.
  param img_width, img_height:   image height and image with scalar values
  return img_width, img_height:  resized image height and image with scalar values     
  """

  if img_width > MAX_WIDTH:
    img_width = MAX_WIDTH

  if img_height > MAX_HEIGHT:
    img_height = MAX_HEIGHT
  
  return img_height, img_width



In [90]:
# set tensorflow dataset parameters
# remove unneccessary files from source directory
train_test_split(origin, train_path, test_split)
#img_width, img_height = get_min_img_size(train_source_folder_path)
#img_width, img_height = set_resize_image_parameters(img_height, img_width)
#image_size = (img_width, img_height)

['5955475577_3d923874d9_n.jpg', '15054750690_198b6ab0f2_n.jpg', '6606813305_c992231d29_m.jpg', '2949654221_909b0c86a1_n.jpg', '3912497870_a2f91c3a65_n.jpg', '1008566138_6927679c8a.jpg', '26254755_1bfc494ef1_n.jpg', '58636535_bc53ef0a21_m.jpg', '2706304885_4916102704_n.jpg', '45045005_57354ee844.jpg', '14925398301_55a180f919_n.jpg', '4623843480_23e3fb8dcc_n.jpg', '3568925290_faf7aec3a0.jpg', '15054865768_2cc87ac9d4_m.jpg', '23894449029_bf0f34d35d_n.jpg', '27465811_9477c9d044.jpg', '21821266773_7113d34c35_m.jpg', '4341530649_c17bbc5d01.jpg', '877083343_e3338c4125.jpg', '20258015499_93b9951800_m.jpg', '15054753070_4f6ae0e763_m.jpg', '21374127408_5ffbe87bb2.jpg', '8563099326_8be9177101.jpg', '20406385204_469f6749e2_n.jpg', '15030133005_9728102622_z.jpg', '4933821940_38064522a8.jpg', '6116210027_61923f4b64.jpg', '14889779907_3d401bbac7_m.jpg', '2598973480_07de93e91d_n.jpg', '20344366953_44fb51051b.jpg', '15495578821_92c6d14252_n.jpg', '200557981_f800fa1af9.jpg', '8014734302_65c6e83bb4_m.jpg

TypeError: ignored

In [68]:
def get_image_dataset(data_path, color_mode, image_size, subset = None, 
                      validation_split = None, seed = None):
  """
  Create tensorflow dataset from data directory object.
  param data_path:        string, source folder path
  param color_mode:       available color modes are "grayscale", "rgb", "rgba", 
                          (images will be converted to have 1, 3, or 4 channels)
  param image_size:       touple (image_with, image_height)
  param subset:           name of subset ('train', 'validation')
  param validation_split: validation set percentage (float)
  """
    
  if subset:
        validation_split = validation_split
  dataset = tf.keras.preprocessing.image_dataset_from_directory(
      data_path,
      color_mode=color_mode,
      image_size=image_size,
      seed=seed,
      validation_split=validation_split, 
      subset=subset
    )
    
  dataset.class_names.sort()
    
  return {
      "data": dataset.cache().prefetch(
      buffer_size = tf.data.experimental.AUTOTUNE
      ),
      "classNames": dataset.class_names
        }

In [72]:

training_ds = get_image_dataset(
    train_source_folder_path,
    color_mode,
    image_size,
    subset = "training",
    validation_split = validation_split, 
    seed=seed
)

validation_ds = get_image_dataset(
    train_source_folder_path,
    color_mode,
    image_size,
    subset = "validation",
    validation_split = validation_split,
    seed=seed
)

"""
test_ds = get_image_dataset(
    test_source_folder_path,
    color_mode,
    image_size
)
"""

Found 3670 files belonging to 6 classes.
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalAnd in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op SelectV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DummySeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV3 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DummyMemoryCache in device /job:localhost/replica:0/tas